# Maîtriser les expressions régulières

Lino Galiana  
2025-10-06

<div class="badge-container"><div class="badge-text">Pour essayer les exemples présents dans ce tutoriel :</div><a href="https://github.com/linogaliana/python-datascientist-notebooks/blob/main/notebooks/manipulation/04b_regex_TP.ipynb" target="_blank" rel="noopener"><img src="https://img.shields.io/static/v1?logo=github&label=&message=View%20on%20GitHub&color=181717" alt="View on GitHub"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/vscode-python?autoLaunch=true&name=«04b_regex_TP»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-vscode.sh»&init.personalInitArgs=«manipulation%2004b_regex_TP%20correction»" target="_blank" rel="noopener"><img src="https://custom-icon-badges.demolab.com/badge/SSP%20Cloud-Lancer_avec_VSCode-blue?logo=vsc&logoColor=white" alt="Onyxia"></a>
<a href="https://datalab.sspcloud.fr/launcher/ide/jupyter-python?autoLaunch=true&name=«04b_regex_TP»&init.personalInit=«https%3A%2F%2Fraw.githubusercontent.com%2Flinogaliana%2Fpython-datascientist%2Fmain%2Fsspcloud%2Finit-jupyter.sh»&init.personalInitArgs=«manipulation%2004b_regex_TP%20correction»" target="_blank" rel="noopener"><img src="https://img.shields.io/badge/SSP%20Cloud-Lancer_avec_Jupyter-orange?logo=Jupyter&logoColor=orange" alt="Onyxia"></a>
<a href="https://colab.research.google.com/github/linogaliana/python-datascientist-notebooks-colab//blob/main//notebooks/manipulation/04b_regex_TP.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a><br></div>

> **Note**
>
> Ceci est la version française 🇫🇷 de ce chapitre, pour voir la version anglaise allez <a href="/home/runner/work/python-datascientist/python-datascientist/en/content/manipulation/04b_regex_TP.qmd">ici</a>.

> **Compétences à l’issue de ce chapitre**
>
> -   Comprendre l’utilité des expressions régulières (regex) pour manipuler des données textuelles de manière flexible, notamment au-delà des limitations de méthodes simples comme `str.find` ;
> -   Maîtriser les concepts fondamentaux des regex : classes de caractères (ex. `[a-z]`), quantifieurs (`?`, `*`, `+`, `{}`), ainsi que l’utilisation d’ancres et de méta-caractères ;
> -   Utiliser les principales fonctions du module `re` de Python (`findall`, `search`, `match`, `sub`, `finditer`, etc.) pour rechercher, extraire ou remplacer des motifs dans des chaînes ;
> -   Apprendre à appliquer efficacement les regex à des données tabulaires en utilisant l’API vectorisée de `Pandas` (`str.contains`, `str.extract`, `str.findall`, `str.replace`, `str.count`) ;
> -   Mettre en pratique les compétences acquises à travers des exercices ciblés : extraction de dates dans une chaîne et extraction d’adresses email ou d’années de publication depuis un DataFrame.

# 1. Introduction

`Python` offre énormément de fonctionalités très pratiques pour la manipulation de données
textuelles. C’est l’une des raisons de son
succès dans la communauté du traitement automatisé du langage (NLP, voir partie dédiée).

Dans les chapitres précédents, nous avons parfois été amenés à chercher des éléments textuels basiques. Cela était possible avec la méthode `str.find` du package `Pandas` qui constitue une version vectorisée de la méthode `find`
de base. Nous avons d’ailleurs
pu utiliser cette dernière directement, notamment lorsqu’on a fait du *web scraping*.

Cependant, cette fonction de recherche
trouve rapidement ses limites.
Par exemple, si on désire trouver à la fois les occurrences d’un terme au singulier
et au pluriel, il sera nécessaire d’utiliser
au moins deux fois la méthode `find`.
Pour des verbes conjugués, cela devient encore plus complexe, en particulier si ceux-ci changent de forme selon le sujet.

Pour des expressions compliquées, il est conseillé d’utiliser les **expressions régulières**,
ou *“regex”*. C’est une fonctionnalité qu’on retrouve dans beaucoup de langages. C’est une forme de grammaire qui permet de rechercher des expressions.

Une partie du contenu de cette partie
est une adaptation de la
[documentation collaborative sur `R` nommée `utilitR`](https://www.book.utilitr.org/03_fiches_thematiques/fiche_donnees_textuelles#regex) à laquelle j’ai participé. Ce chapitre reprend aussi du contenu du
livre [*R for Data Science*](https://r4ds.hadley.nz/regexps.html) qui présente un chapitre
très pédagogique sur les regex.

Nous allons utiliser le *package* `re` pour illustrer nos exemples d’expressions
régulières. Il s’agit du package de référence, qui est utilisé, en arrière-plan,
par `Pandas` pour vectoriser les recherches textuelles.

In [ ]:
import re
import pandas as pd

> **Tip**
>
> **Les expressions régulières (*regex*) sont notoirement difficiles à maîtriser.** Il existe des outils qui facilitent le travail avec les expressions régulières.
>
> -   L’outil de référence pour ceci est \[https://regex101.com/\] qui permet de tester des `regex` en `Python`
>     tout en ayant une explication qui accompagne ce test
>
> -   De même pour [ce site](https://ole.michelsen.dk/tools/regex/) qui comporte une cheat sheet en bas de la page.
>
> -   Les jeux de [Regex Crossword](https://regexcrossword.com/) permettent d’apprendre les expressions régulières en s’amusant
>
> Il peut être pratique de demander à des IA assistantes, comme `Github Copilot` ou `ChatGPT`, une
> première version d’une regex en expliquant le contenu qu’on veut extraire.
> Cela peut faire économiser pas mal de temps, sauf quand l’IA fait preuve d’une confiance excessive
> et vous propose avec aplomb une regex totalement fausse…

# 2. Principe

**Les expressions régulières sont un outil permettant de décrire un ensemble de chaînes de caractères possibles selon une syntaxe précise, et donc de définir un motif (ou `pattern`).** Les expressions régulières servent par exemple lorsqu’on veut extraire une partie d’une chaîne de caractères, ou remplacer une partie d’une chaîne de caractères. Une expression régulière prend la forme d’une chaîne de caractères, qui peut contenir à la fois des éléments littéraux et des caractères spéciaux qui ont un sens logique.

Par exemple, `"ch.+n"` est une expression régulière qui décrit le motif suivant : la chaîne littérale `ch`, suivi de n’importe quelle chaîne d’au moins un caractère (`.+`), suivie de la lettre `n`. Dans la chaîne `"J'ai un chien."`, la sous-chaîne `"chien"` correspond à ce motif. De même pour `"chapeau ron"` dans `"J'ai un chapeau rond"`. En revanche, dans la chaîne `"La soupe est chaude."`, aucune sous-chaîne ne correpsond à ce motif (car aucun `n` n’apparaît après le `ch`).

Pour s’en convaincre, nous pouvons déjà regarder
les deux premiers cas:

In [ ]:
pattern = "ch.+n"
print(re.search(pattern, "La soupe est chaude."))

None

La regex précédente comportait deux types de caractères:

-   les *caractères littéraux* : lettres et nombres qui sont reconnus de manière littérale
-   les *méta-caractères* : symboles qui ont un sens particulier dans les regex.

Les principaux *méta-caractères* sont `.`, `+`, `*`, `[`, `]`, `^` et `$` mais il
en existe beaucoup d’autres.
Parmi cet ensemble, on utilise principalement les quantifieurs (`.`, `+`, `*`…),
les classes de caractères (ensemble qui sont délimités par `[` et `]`)
ou les ancres (`^`, `$`…)

Dans l’exemple précédent,
nous retrouvions deux quantifieurs accolés `.+`. Le premier (`.`) signifie n’importe quel caractère[1]. Le deuxième (`+`) signifie *“répète le pattern précédent”*.
Dans notre cas, la combinaison `.+` permet ainsi de répéter n’importe quel caractère avant de trouver un *n*.
Le nombre de fois est indeterminé : cela peut ne pas être pas nécessaire d’intercaler des caractères avant le *n*
ou cela peut être nécessaire d’en intercepter plusieurs :

[1] N’importe quel caractère à part le retour à la ligne (`\n`). Ceci est à garder en tête, j’ai déjà perdu des heures à chercher pourquoi mon `.` ne capturait pas ce que je voulais qui s’étalait sur plusieurs lignes…

In [ ]:
print(re.search(pattern, "J'ai un chino"))
print(re.search(pattern, "J'ai un chiot très mignon."))

<re.Match object; span=(8, 12), match='chin'>
<re.Match object; span=(8, 25), match='chiot très mignon'>

## 2.1 Classes de caractères

Lors d’une recherche, on s’intéresse aux caractères et souvent aux classes de caractères : on cherche un chiffre, une lettre, un caractère dans un ensemble précis ou un caractère qui n’appartient pas à un ensemble précis. Certains ensembles sont prédéfinis, d’autres doivent être définis à l’aide de crochets.

Pour définir un ensemble de caractères, il faut écrire cet ensemble entre crochets. Par exemple, `[0123456789]` désigne un chiffre. Comme c’est une séquence de caractères consécutifs, on peut résumer cette écriture en `[0-9]`.

Par
exemple, si on désire trouver tous les *pattern* qui commencent par un `c` suivi
d’un `h` puis d’une voyelle (a, e, i, o, u), on peut essayer
cette expression régulière.

In [ ]:
re.findall("[c][h][aeiou]", "chat, chien, veau, vache, chèvre")

['cha', 'chi', 'che']

Il serait plus pratique d’utiliser `Pandas` dans ce cas pour isoler les
lignes qui répondent à la condition logique (en ajoutant les accents
qui ne sont pas compris sinon):

In [ ]:
import pandas as pd
txt = pd.Series("chat, chien, veau, vache, chèvre".split(", "))
txt.str.match("ch[aeéèiou]")

0     True
1     True
2    False
3    False
4     True
dtype: bool

Cependant, l’usage ci-dessus des classes de caractères
n’est pas le plus fréquent.
On privilégie celles-ci pour identifier des
pattern complexe plutôt qu’une suite de caractères littéraux.
Les tableaux d’aide mémoire illustrent une partie des
classes de caractères les plus fréquentes
(`[:digit:]` ou `\d`…)

## 2.2 Quantifieurs

Nous avons rencontré les quantifieurs avec notre première expression
régulière. Ceux-ci contrôlent le nombre de fois
qu’un *pattern* est rencontré.

Les plus fréquents sont:

-   `?` : 0 ou 1 match ;
-   `+` : 1 ou plus de matches ;
-   `*` : 0 or more matches.

Par exemple, `colou?r` permettra de matcher à la fois l’écriture américaine et anglaise

In [ ]:
re.findall("colou?r", "Did you write color or colour?")

['color', 'colour']

Ces quantifiers peuvent bien sûr être associés à
d’autres types de caractères, notamment les classes de caractères.
Cela peut être extrêmement pratique.
Par exemple, `\d+` permettra de capturer un ou plusieurs chiffres, `\s?`
permettra d’ajouter en option un espace,
`[\w]{6,8}` un mot entre six et huit lettres qu’on écrira…

Il est aussi possible de définir le nombre de répétitions
avec `{}`:

-   `{n}` matche exactement *n* fois ;
-   `{n,}` matche au moins *n* fois ;
-   `{n,m}` matche entre *n* et *m* fois.

Cependant, la répétition des termes
ne s’applique par défaut qu’au dernier
caractère précédent le quantifier.
On peut s’en convaincre avec l’exemple ci-dessus:

In [ ]:
print(re.match("toc{4}","toctoctoctoc"))

None

Pour pallier ce problème, il existe les parenthèses.
Le principe est le même qu’avec les règles numériques:
les parenthèses permettent d’introduire une hiérarchie.
Pour reprendre l’exemple précédent, on obtient
bien le résultat attendu grâce aux parenthèses:

In [ ]:
print(re.match("(toc){4}","toctoctoctoc"))
print(re.match("(toc){5}","toctoctoctoc"))
print(re.match("(toc){2,4}","toctoctoctoc"))

<re.Match object; span=(0, 12), match='toctoctoctoc'>
None
<re.Match object; span=(0, 12), match='toctoctoctoc'>

> **Note**
>
> L’algorithme des expressions régulières essaye toujours de faire correspondre le plus grand morceau à l’expression régulière.
>
> Par exemple, soit une chaine de caractère HTML:
>
> ``` python
> s = "<h1>Super titre HTML</h1>"
> ```
>
> L’expression régulière `re.findall("<.*>", s)` correspond, potentiellement,
> à trois morceaux :
>
> -   `<h1>`
> -   `</h1>`
> -   `<h1>Super titre HTML</h1>`
>
> C’est ce dernier qui sera choisi, car le plus grand. Pour
> sélectionner le plus petit,
> il faudra écrire les multiplicateurs comme ceci : `*?`, `+?`.
> En voici quelques exemples:
>
> ``` python
> s = "<h1>Super titre HTML</h1>\n<p><code>Python</code> est un langage très flexible</p>"
> print(re.findall("<.*>", s))
> print(re.findall("<p>.*</p>", s))
> print(re.findall("<p>.*?</p>", s))
> print(re.compile("<.*?>").findall(s))
> ```
>
>     ['<h1>Super titre HTML</h1>', '<p><code>Python</code> est un langage très flexible</p>']
>     ['<p><code>Python</code> est un langage très flexible</p>']
>     ['<p><code>Python</code> est un langage très flexible</p>']
>     ['<h1>', '</h1>', '<p>', '<code>', '</code>', '</p>']

## 2.3 Aide-mémoire

Le tableau ci-dessous peut servir d’aide-mémoire
sur les regex:

| Expression régulière | Signification |
|--------------------------|----------------------------------------------|
| `"^"` | Début de la chaîne de caractères |
| `"$"` | Fin de la chaîne de caractères |
| `"\\."` | Un point |
| `"."` | N’importe quel caractère |
| `".+"` | N’importe quelle suite de caractères non vide |
| `".*"` | N’importe quelle suite de caractères, éventuellement vi |
| `"[:alnum:]"` | Un caractère alphanumérique |
| `"[:alpha:]"` | Une lettre |
| `"[:digit:]"` | Un chiffre |
| `"[:lower:]"` | Une lettre minuscule |
| `"[:punct:]"` | Un signe de ponctuation |
| `"[:space:]"` | un espace |
| `"[:upper:]"` | Une lettre majuscule |
| `"[[:alnum:]]+"` | Une suite d’au moins un caractère alphanumérique |
| `"[[:alpha:]]+"` | Une suite d’au moins une lettre |
| `"[[:digit:]]+"` | Une suite d’au moins un chiffre |
| `"[[:lower:]]+"` | Une suite d’au moins une lettre minuscule |
| `"[[:punct:]]+"` | Une suite d’au moins un signe de ponctuation |
| `"[[:space:]]+"` | Une suite d’au moins un espace |
| `"[[:upper:]]+"` | Une suite d’au moins une lettre majuscule |
| `"[[:alnum:]]*"` | Une suite de caractères alphanumériques, éventuellement vide |
| `"[[:alpha:]]*"` | Une suite de lettres, éventuellement vide |
| `"[[:digit:]]*"` | Une suite de chiffres, éventuellement vide |
| `"[[:lower:]]*"` | Une suite de lettres minuscules, éventuellement vide |
| `"[[:upper:]]*"` | Une suite de lettres majuscules, éventuellement vide |
| `"[[:punct:]]*"` | Une suite de signes de ponctuation, éventuellement vide |
| `"[^[:alpha:]]+"` | Une suite d’au moins un caractère autre qu’une lettre |
| `"[^[:digit:]]+"` | Une suite d’au moins un caractère autre qu’un chiffre |
| `"\|"` | L’une des expressions `x` ou `y` est présente |
| `[abyz]` | Un seul des caractères spécifiés |
| `[abyz]+` | Un ou plusieurs des caractères spécifiés (éventuellement répétés) |
| `[^abyz]` | Aucun des caractères spécifiés n’est présent |

Certaines classes de caractères bénéficient d’une syntaxe plus légère car
elles sont très fréquentes. Parmi-celles:

| Expression régulière | Signification |
|--------------------------|----------------------------------------------|
| `\d` | N’importe quel chiffre |
| `\D` | N’importe quel caractère qui n’est pas un caractère |
| `\s` | N’importe quel espace (espace, tabulation, retour à la ligne) |
| `\S` | N’importe quel caractère qui n’est pas un espace |
| `\w` | N’importe quel type de mot (lettres et nombres) |
| `\W` | N’importe quel ensemble qui n’est pas un mot (lettres et nombres) |

Dans l’exercice suivant, vous allez pouvoir mettre en pratique
les exemples précédents sur une `regex` un peu plus complète.
Cet exercice ne nécessite pas la connaissance des subtilités
du *package* `re`, vous n’aurez besoin que de `re.findall`.

Cet exercice utilisera la chaine de caractère suivante :

In [ ]:
s = """date 0 : 14/9/2000
date 1 : 20/04/1971     date 2 : 14/09/1913     date 3 : 2/3/1978
date 4 : 1/7/1986     date 5 : 7/3/47     date 6 : 15/10/1914
date 7 : 08/03/1941     date 8 : 8/1/1980     date 9 : 30/6/1976"""
s

'date 0 : 14/9/2000\ndate 1 : 20/04/1971     date 2 : 14/09/1913     date 3 : 2/3/1978\ndate 4 : 1/7/1986     date 5 : 7/3/47     date 6 : 15/10/1914\ndate 7 : 08/03/1941     date 8 : 8/1/1980     date 9 : 30/6/1976'

> **Exercice 1**
>
> 1.  On va d’abord s’occuper d’extraire le jour de naissance.
>     -   Le premier chiffre du jour est 0, 1, 2 ou 3. Traduire cela sous la forme d’une séquence `[X-X]`
>     -   Le deuxième chiffre du jour est lui entre 0 et 9. Traduire cela sous la séquence adéquate
>     -   Remarquez que le premier jour est facultatif. Intercaler entre les deux classes de caractère adéquate
>         le quantifieur qui convient
>     -   Ajouter le slash à la suite du motif
>     -   Tester avec `re.findall`. Vous devriez obtenir beaucoup plus d’échos que nécessaire.
>         C’est normal, à ce stade la
>         regex n’est pas encore finalisée
> 2.  Suivre la même logique pour les mois en notant que les mois du calendrier grégorien ne dépassent
>     jamais la première dizaine. Tester avec `re.findall`
> 3.  De même pour les années de naissance en notant que jusqu’à preuve du contraire, pour des personnes vivantes
>     aujourd’hui, les millénaires concernés sont restreints. Tester avec `re.findall`
> 4.  Cette regex n’est pas naturelle, on pourrait très bien se satisfaire de classes de
>     caractères génériques `\d` même si elles pourraient, en pratique, nous sélectionner des
>     dates de naissance non possibles (`43/78/4528` par exemple). Cela permettrait
>     d’alléger la regex afin de la rendre plus intelligible. Ne pas oublier l’utilité des quantifieurs.
> 5.  Comment adapter la regex pour qu’elle soit toujours valide pour nos cas mais permette aussi de
>     capturer les dates de type `YYYY/MM/DD` ? Tester sur `1998/07/12`

A l’issue de la question 1, vous devriez avoir ce résultat :

A l’issue de la question 2, vous devriez avoir ce résultat, qui commence à prendre forme:

A l’issue de la question 3, on parvient bien à extraire les dates :

Si tout va bien, à la question 5, votre regex devrait fonctionner:

# 3. Principales fonctions de `re`

Voici un tableau récapitulatif des principales
fonctions du package `re` suivi d’exemples.

Nous avons principalement
utilisé jusqu’à présent `re.findall` qui est
l’une des fonctions les plus pratiques du *package*.
`re.sub` et `re.search` sont également bien pratiques.
Les autres sont moins vitales mais peuvent dans des
cas précis être utiles.

| Fonction | Objectif |
|-------------------------------------|-----------------------------------|
| `re.match(<regex>, s)` | Trouver et renvoyer le **premier** *match* de l’expression régulière `<regex>` **à partir du début** du *string* `s` |
| `re.search(<regex>, s)` | Trouver et renvoyer le **premier** *match* de l’expression régulière `<regex>` **quelle que soit sa position** dans le *string* `s` |
| `re.finditer(<regex>, s)` | Trouver et renvoyer un itérateur stockant tous les *matches* de l’expression régulière `<regex>` **quelle que soit leur(s) position(s)** dans le *string* `s`. En général, on effectue ensuite une boucle sur cet itérateur |
| `re.findall(<regex>, s)` | Trouver et renvoyer **tous les *matches*** de l’expression régulière `<regex>` **quelle que soit leur(s) position(s)** dans le *string* `s` sous forme de **liste** |
| `re.sub(<regex>, new_text, s)` | Trouver et **remplacer tous** les *matches* de l’expression régulière `<regex>` **quelle que soit leur(s) position(s)** dans le *string* `s` |

Pour illustrer ces fonctions, voici quelques exemples:

<details><summary>Exemple de <code>re.match</code> 👇</summary>

`re.match` ne peut servir qu’à capturer un *pattern* en début
de *string*. Son utilité est donc limitée.
Capturons néanmoins `toto` :

``` python
re.match("(to){2}", "toto à la plage")
```

    <re.Match object; span=(0, 4), match='toto'>

</details>

<details><summary>Exemple de <code>re.search</code> 👇</summary>

`re.search` est plus puissant que `re.match`, on peut
capturer des termes quelle que soit leur position
dans un *string*. Par exemple, pour capturer *age* :

``` python
re.search("age", "toto a l'age d'aller à la plage")
```

    <re.Match object; span=(9, 12), match='age'>

Et pour capturer exclusivement *“age”* en fin
de *string* :

``` python
re.search("age$", "toto a l'age d'aller à la plage")
```

    <re.Match object; span=(28, 31), match='age'>

</details>

<details><summary>Exemple de <code>re.finditer</code> 👇</summary>

`re.finditer` est, à mon avis,
moins pratique que `re.findall`. Son utilité
principale par rapport à `re.findall`
est de capturer la position dans un champ textuel:

``` python
s = "toto a l'age d'aller à la plage"
for match in re.finditer("age", s):
    start = match.start()
    end = match.end()
    print(f'String match "{s[start:end]}" at {start}:{end}')
```

    String match "age" at 9:12
    String match "age" at 28:31

</details>

<details><summary>Exemple de <code>re.sub</code> 👇</summary>

`re.sub` permet de capturer et remplacer des expressions.
Par exemple, remplaçons *“age”* par *“âge”*. Mais attention,
il ne faut pas le faire lorsque le motif est présent dans *“plage”*.
On va donc mettre une condition négative: capturer *“age”* seulement
s’il n’est pas en fin de *string* (ce qui se traduit en *regex* par `?!$`)

``` python
re.sub("age(?!$)", "âge", "toto a l'age d'aller à la plage")
```

    "toto a l'âge d'aller à la plage"

</details>

> **Quand utiliser `re.compile` et les raw strings ?**
>
> `re.compile` peut être intéressant lorsque
> vous utilisez une expression régulière plusieurs fois dans votre code.
> Cela permet de compiler l’expression régulière en un objet reconnu par `re`,
> ce qui peut être plus efficace en termes de performance lorsque l’expression régulière
> est utilisée à plusieurs reprises ou sur des données volumineuses.
>
> Les chaînes brutes (*raw string*) sont des chaînes de caractères spéciales en `Python`,
> qui commencent par `r`. Par exemple `r"toto à la plage"`.
> Elles peuvent être intéressantes
> pour éviter que les caractères d’échappement ne soient interprétés par `Python`
> Par exemple, si vous voulez chercher une chaîne qui contient une barre oblique inverse `\` dans une chaîne, vous devez utiliser une chaîne brute pour éviter que la barre oblique inverse ne soit interprétée comme un caractère d’échappement (`\t`, `\n`, etc.).
> Le testeur <https://regex101.com/> suppose d’ailleurs que
> vous utilisez des *raw string*, cela peut donc être utile de s’habituer à les utiliser.

# 4. Généralisation avec `Pandas`

Les méthodes de `Pandas` sont des extensions de celles de `re`
qui évitent de faire une boucle pour regarder,
ligne à ligne, une regex. En pratique, lorsqu’on traite des
`DataFrames`, on utilise plutôt l’API Pandas que `re`. Les
codes de la forme `df.apply(lambda x: re.<fonction>(<regex>,x), axis = 1)`
sont à bannir car très peu efficaces.

Les noms changent parfois légèrement par rapport à leur
équivalent `re`.

| Méthode | Description |
|---------------------------------------|---------------------------------|
| `str.count()` | Compter le nombre d’occurrences du *pattern* dans chaque ligne |
| `str.replace()` | Remplacer le *pattern* par une autre valeur. Version vectorisée de `re.sub()` |
| `str.contains()` | Tester si le *pattern* apparaît, ligne à ligne. Version vectorisée de `re.search()` |
| `str.extract()` | Extraire les groupes qui répondent à un *pattern* et les renvoyer dans une colonne |
| `str.findall()` | Trouver et renvoyer toutes les occurrences d’un *pattern*. Si une ligne comporte plusieurs échos, une liste est renvoyée. Version vectorisée de `re.findall()` |

A ces fonctions, s’ajoutent les méthodes `str.split()` et `str.rsplit()` qui sont bien pratiques.

<details><summary>Exemple de <code>str.count</code> 👇</summary>

On peut compter le nombre de fois qu’un *pattern* apparaît avec
`str.count`

``` python
df = pd.DataFrame({"a": ["toto", "titi"]})
df['a'].str.count("to")
```

    0    2
    1    0
    Name: a, dtype: int64

</details>

<details><summary>Exemple de <code>str.replace</code> 👇</summary>

Remplaçons le motif *“ti”* en fin de phrase

``` python
df = pd.DataFrame({"a": ["toto", "titi"]})
df['a'].str.replace("ti$", " punch")
```

    0    toto
    1    titi
    Name: a, dtype: object

</details>

<details><summary>Exemple de <code>str.contains</code> 👇</summary>

Vérifions les cas où notre ligne termine par *“ti”* :

``` python
df = pd.DataFrame({"a": ["toto", "titi"]})
df['a'].str.contains("ti$")
```

    0    False
    1     True
    Name: a, dtype: bool

</details>

<details><summary>Exemple de <code>str.findall</code> 👇</summary>

``` python
df = pd.DataFrame({"a": ["toto", "titi"]})
df['a'].str.findall("to")
```

    0    [to, to]
    1          []
    Name: a, dtype: object

</details>

# 5. Pour en savoir plus

-   [documentation collaborative sur `R` nommée `utilitR`](https://www.book.utilitr.org/03_fiches_thematiques/fiche_donnees_textuelles#regex)
-   [*R for Data Science*](https://r4ds.hadley.nz/regexps.html)
-   [*Regular Expression HOWTO* dans la documentation officielle de `Python`](https://docs.python.org/3/howto/regex.html)
-   L’outil de référence \[https://regex101.com/\] pour tester des expressions régulières
-   [Ce site](https://ole.michelsen.dk/tools/regex/) qui comporte une cheat sheet en bas de la page.
-   Les jeux de [Regex Crossword](https://regexcrossword.com/) permettent d’apprendre les expressions régulières en s’amusant

# 6. Exercices supplémentaires

## 6.1 Extraction d’adresses email

Il s’agit d’un usage classique des *regex*

In [ ]:
text_emails = 'Hello from toto@gmail.com to titi.grominet@yahoo.com about the meeting @2PM'

> **Exercice 2: extraction d’adresses email**
>
> Utiliser la structure d’une adresse mail `[XXXX]@[XXXX]` pour récupérer
> ce contenu

## 6.2 Extraire des années depuis un `DataFrame` `Pandas`

L’objectif général de l’exercice est de nettoyer des colonnes d’un DataFrame en utilisant des expressions régulières.

> **Exercice 3**
>
> La base en question contient des livres de la British Library et quelques informations les concernant. Le jeu de données est disponible ici : https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/BL-Flickr-Images-Book.csv
>
> La colonne “Date de Publication” n’est pas toujours une année, il y a parfois d’autres informations. Le but de l’exercice est d’avoir **une date de publication du livre propre** et de regarder la **distribution des années de publications**.
>
> Pour ce faire, vous pouvez :
>
> -   Soit choisir de réaliser l’exercice sans aide. Votre **lecture de l’énoncé s’arrête donc ici**. Vous devez alors faire attention à bien regarder vous-même la base de données et la transformer avec attention.
>
> -   Soit suivre les différentes étapes qui suivent pas à pas.
>
> <details>
>
> <summary>
>
> Version guidée 👇
>
> </summary>
>
> 1.  Lire les données depuis l’url `https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/BL-Flickr-Images-Book.csv`. Attention au séparateur
> 2.  Ne garder que les colonnes `['Identifier', 'Place of Publication', 'Date of Publication', 'Publisher', 'Title', 'Author']`
> 3.  Observer la colonne *‘Date of Publication’* et remarquer le problème sur certaines lignes (par exemple la ligne 13)
> 4.  Commencez par regarder le nombre d’informations manquantes. On ne pourra pas avoir mieux après la regex, et normalement on ne devrait pas avoir moins…
> 5.  Déterminer la forme de la regex pour une date de publication. A priori, il y a 4 chiffres qui forment une année.
>     Utiliser la méthode `str.extract()` avec l’argument `expand = False` (pour ne conserver que la première date concordant avec notre *pattern*)?
> 6.  On a 2 `NaN` qui n’étaient pas présents au début de l’exercice. Quels sont-ils et pourquoi ?
> 7.  Quelle est la répartition des dates de publications dans le jeu de données ? Vous pouvez par exemple afficher un histogramme grâce à la méthode `plot` avec l’argument `kind ="hist"`.
>
> </details>

Voici par exemple le problème qu’on demande de détecter à la question 3 :

A la question 4, on obtient la réponse

Grâce à notre regex (question 5), on obtient ainsi un `DataFrame` plus conforme à nos attentes

Quant aux nouveaux `NaN`,
il s’agit de lignes qui ne contenaient pas de chaînes de caractères qui ressemblaient à des années :

Enfin, on obtient l’histogramme suivant des dates de publications: